In [1]:
import pandas as pd
from rdkit import Chem
from glob import glob
import os
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.core import Molecule, Site

In [43]:
def parse_positions(pos_lines, opt_coords_start, parser="qchem"):
    # helper function to parse final positions from qchem log file
    dict_return = {}

    # remove all lines before opt_coords_start
    pos_lines = pos_lines[opt_coords_start:]
    # find line that starts with ----
    if parser == "qchem":
        lines_coord_block = [
            i for i, line in enumerate(pos_lines) if line.strip().startswith("----")
        ]
        start_ind = lines_coord_block[0]
        end_ind = lines_coord_block[1]

    elif parser == "molpro":
        # look for empty line
        lines_coord_block = [
            i for i, line in enumerate(pos_lines) if line.strip() == ""
        ]
        start_ind = lines_coord_block[1]
        end_ind = lines_coord_block[2]

    # remove all lines after end_ind
    pos_lines = pos_lines[start_ind + 1 : end_ind]

    for atom in pos_lines:
        atom = atom.split()
        # print(atom[0])
        if parser == "qchem":
            dict_return[str(int(atom[0]) - 1)] = {
                "element": atom[1],
                "pos": [float(i) for i in atom[2:]],
            }

        elif parser == "molpro":
            dict_return[str(int(atom[0]) - 1)] = {
                "element": atom[1],
                "pos": [float(i) for i in atom[3:]],
            }

    return dict_return


def get_positions_from_log(
    df_row, prefix_log, log_folder_root, zero_padding_folder,  parser="qchem"
):
    ## helper function that's more zoomed out that parse_positions, handles multiple products and reactants.

    r_dict_list = []
    p_dict_list = []

    ind = df_row.idx

    product_string = (
        log_folder_root + prefix_log + str(ind).zfill(zero_padding_folder) + "/p*"
    )
    reactant_string = (
        log_folder_root + prefix_log + str(ind).zfill(zero_padding_folder) + "/r*"
    )

    #print(product_string)
    #print(reactant_string)
    product_files = glob(product_string)
    reactant_files = glob(reactant_string)
    # sort files by alphabetical order
    product_files.sort()
    reactant_files.sort()
    print("number of log files found:", len(product_files) + len(reactant_files))

    for product_file in product_files:
        # open file, readlines
        product_lines = open(product_file).readlines()
        if parser == "qchem":
            p_lines_coord_block = [
                i
                for i, line in enumerate(product_lines)
                if line.strip().startswith("Standard Nuclear Orientation (Angstroms)")
            ]
        elif parser == "molpro":
            p_lines_coord_block = [
                i
                for i, line in enumerate(product_lines)
                if line.strip().startswith("ATOMIC COORDINATES")
            ]
        p_opt_coords_start = p_lines_coord_block[-1]
        p_dict = parse_positions(product_lines, p_opt_coords_start, parser=parser)
        p_dict_list.append(p_dict)

    for reactant_file in reactant_files:
        reactant_lines = open(reactant_file).readlines()

        # find line that starts with "Standard Nuclear Orientation (Angstroms)"
        if parser == "qchem":
            r_lines_coord_block = [
                i
                for i, line in enumerate(reactant_lines)
                if line.strip().startswith("Standard Nuclear Orientation (Angstroms)")
            ]
        elif parser == "molpro":
            r_lines_coord_block = [
                i
                for i, line in enumerate(reactant_lines)
                if line.strip().startswith("ATOMIC COORDINATES")
            ]

        r_opt_coords_start = r_lines_coord_block[-1]

        r_dict = parse_positions(reactant_lines, r_opt_coords_start, parser=parser)
        r_dict_list.append(r_dict)
    return r_dict_list, p_dict_list


def get_bonds_list_dict(
    df_row, prefix_log, log_folder_root, zero_padding_folder, parser="qchem"
):
    # helper to get bond list from positions, does use rdkit so only works for organic molecules-ish
    r_atoms_count = 0
    p_atoms_count = 0
    r_dict_list, p_dict_list = get_positions_from_log(
        df_row, prefix_log, log_folder_root, zero_padding_folder, parser=parser
    )

    for r_dict in r_dict_list:
        r_atoms_count += len(r_dict)
    for p_dict in p_dict_list:
        p_atoms_count += len(p_dict)

    assert (
        r_atoms_count == p_atoms_count
    ), "r and p dict should have same length, they are {} and {} for rxn w id {}".format(
        r_atoms_count, p_atoms_count, df_row.idx
    )
    # write both to temp xyz files
    with open("p.xyz", "w") as f:
        f.write(str(len(p_dict)) + "\n")
        f.write("comment\n")
        for key, val in p_dict.items():
            f.write(
                f"{val['element']} {val['pos'][0]} {val['pos'][1]} {val['pos'][2]}\n"
            )
    with open("r.xyz", "w") as f:
        f.write(str(len(r_dict)) + "\n")
        f.write("comment\n")
        for key, val in r_dict.items():
            f.write(
                f"{val['element']} {val['pos'][0]} {val['pos'][1]} {val['pos'][2]}\n"
            )

    os.system("obabel -ixyz p.xyz -osdf -O p.sdf")
    os.system("obabel -ixyz r.xyz -osdf -O r.sdf")
    # read both to sdf structures
    p_mol = Chem.SDMolSupplier(
        "p.sdf", removeHs=False, sanitize=False, strictParsing=False
    )[0]
    r_mol = Chem.SDMolSupplier(
        "r.sdf", removeHs=False, sanitize=False, strictParsing=False
    )[0]

    # r_mol = Chem.MolFromSmiles(df_row.rsmi)
    r_bond_list = r_mol.GetBonds()
    p_bond_list = p_mol.GetBonds()
    # set atomic indices to be the same as in the sdf file
    for atom in r_mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx() + 1)
    for atom in p_mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx() + 1)

    r_bond_list = [
        [bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()] for bond in r_bond_list
    ]

    p_bond_list = [
        [bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()] for bond in p_bond_list
    ]

    # sort bond list in each
    r_bond_list = [sorted(bond) for bond in r_bond_list]
    p_bond_list = [sorted(bond) for bond in p_bond_list]

    return r_bond_list, p_bond_list, r_mol, p_mol


def get_disjoint_mol_map(df_row):
    # helper to merge sites from different log files
    # relies on mapped smiles from green dataset

    # get indices of atoms in each molecule
    split_products = df_row.psmi.split(".")
    split_reactants = df_row.rsmi.split(".")
    # split_a = df_row.psmi.split(".").split("[")
    split_prod_a = [i.split("[") for i in split_products]
    split_react_a = [i.split("[") for i in split_reactants]
    # remove '' from list
    split_prod_a = [[i for i in j if i != ""] for j in split_prod_a]
    split_react_a = [[i for i in j if i != ""] for j in split_react_a]
    ret_map_products = []
    ret_map_reactants = []

    for i in split_prod_a:
        single_map = []
        for sub_i in i:
            sanitize = sub_i.split("]")
            sanitize = [i for i in sanitize if ":" in i]
            sanitize = [int(i.split(":")[1]) for i in sanitize]
            if len(sanitize) > 0:
                # print("sanitze", sanitize)
                single_map.append(sanitize[0])
        ret_map_products.append(single_map)

    for i in split_react_a:
        single_map = []
        for sub_i in i:
            sanitize = sub_i.split("]")
            sanitize = [i for i in sanitize if ":" in i]
            sanitize = [int(i.split(":")[1]) for i in sanitize]
            # sanitize = [i[0] for i in sanitize]
            # print(sanitize[0])
            if len(sanitize) > 0:
                single_map.append(sanitize[0])
        ret_map_reactants.append(single_map)

    return ret_map_reactants, ret_map_products


def get_global_bond_list(df_row):
    # helper to convert bond list to global indices

    p_mol = Chem.MolFromSmiles(df_row.psmi, sanitize=False)
    r_mol = Chem.MolFromSmiles(df_row.rsmi, sanitize=False)

    r_bond_list = r_mol.GetBonds()
    p_bond_list = p_mol.GetBonds()
    r_atoms = r_mol.GetAtoms()
    p_atoms = p_mol.GetAtoms()

    # for atom in r_atoms:
    #    print(atom.GetSymbol(), atom.GetIdx(), atom.GetAtomMapNum())

    r_bond_list = [
        [
            r_atoms[bond.GetBeginAtomIdx()].GetAtomMapNum() - 1,
            r_atoms[bond.GetEndAtomIdx()].GetAtomMapNum() - 1,
        ]
        for bond in r_bond_list
    ]

    p_bond_list = [
        [
            p_atoms[bond.GetBeginAtomIdx()].GetAtomMapNum() - 1,
            p_atoms[bond.GetEndAtomIdx()].GetAtomMapNum() - 1,
        ]
        for bond in p_bond_list
    ]

    r_bond_list = [sorted(bond) for bond in r_bond_list]
    p_bond_list = [sorted(bond) for bond in p_bond_list]
    bonds_formed = [bond for bond in p_bond_list if bond not in r_bond_list]
    bonds_broken = [bond for bond in r_bond_list if bond not in p_bond_list]

    return r_bond_list, p_bond_list, bonds_broken, bonds_formed


def construct_pmg(
    df_row,
    r_bond_list,
    p_bond_list,
    prefix_log,
    log_folder_root,
    zero_padding_folder,
    parser="qchem",
    single_file=False
):
    # helper to construct pmg molecule graph from bond list + positions/elements
    reactant_unified_dict, product_unified_dict = merge_map_dict(
        df_row,
        prefix_log,
        log_folder_root,
        zero_padding_folder,
        parser=parser,
        single_file=single_file,
    )
    assert len(reactant_unified_dict) == len(product_unified_dict), "need same length"

    # [print(i) for i in reactant_unified_dict]
    r_pos = [i["pos"] for i in list(reactant_unified_dict.values())]
    p_pos = [i["pos"] for i in list(product_unified_dict.values())]

    r_species = [i["element"] for i in list(reactant_unified_dict.values())]
    p_species = [i["element"] for i in list(product_unified_dict.values())]

    r_molecule = Molecule(species=r_species, coords=r_pos)
    p_molecule = Molecule(species=p_species, coords=p_pos)

    r_molecule_graph = MoleculeGraph.with_empty_graph(r_molecule)
    p_molecule_graph = MoleculeGraph.with_empty_graph(p_molecule)

    [r_molecule_graph.add_edge(bond[0], bond[1]) for bond in r_bond_list]
    [p_molecule_graph.add_edge(bond[0], bond[1]) for bond in p_bond_list]

    return r_molecule_graph, p_molecule_graph


def merge_map_dict(
    df_row,
    prefix_log,
    log_folder_root,
    zero_padding_folder,
    parser="qchem",
    single_file=False,
):
    # helper to merge positions from different log files
    product_unified_dict = {}
    reactant_unified_dict = {}
    translation_distance = 25.0
    r_atoms_count, p_atoms_count = 0, 0
    r_dict_list, p_dict_list = get_positions_from_log(
        df_row, prefix_log, log_folder_root, zero_padding_folder, parser=parser
    )

    for r_dict in r_dict_list:
        r_atoms_count += len(r_dict)
    for p_dict in p_dict_list:
        p_atoms_count += len(p_dict)

    # print("mappings")
    # print(map_prod)
    # print(p_dict_list)
    # print(map_react)
    # print(r_dict_list)

    if not single_file:
        map_react, map_prod = get_disjoint_mol_map(df_row)

        assert len(map_react) == len(
            r_dict_list
        ), "map react and r dict list should have same length, failed for rxn w id {}".format(
            df_row.idx
        )
        assert len(map_prod) == len(
            p_dict_list
        ), "map prod and p dict list should have same length, failed for rxn w id {}".format(
            df_row.idx
        )

        assert (
            r_atoms_count == p_atoms_count
        ), "r and p dict should have same length, they are {} and {} for rxn w id {}".format(
            r_atoms_count, p_atoms_count, df_row.idx
        )

        if len(r_dict_list) > 1:
            for ind, sub_dict in enumerate(r_dict_list):
                for key, val in sub_dict.items():
                    val["pos"][0] += translation_distance * ind
                    val["pos"][1] += translation_distance * ind
                    val["pos"][2] += translation_distance * ind

        if len(p_dict_list) > 1:
            for ind, sub_dict in enumerate(p_dict_list):
                for key, val in sub_dict.items():
                    val["pos"][0] += translation_distance * ind
                    val["pos"][1] += translation_distance * ind
                    val["pos"][2] += translation_distance * ind

        for ind_species, specie in enumerate(map_react):
            for ind, atom_global in enumerate(specie):
                reactant_unified_dict[str(atom_global)] = r_dict_list[ind_species][
                    str(ind)
                ]

        for ind_species, specie in enumerate(map_prod):
            for ind, atom_global in enumerate(specie):
                product_unified_dict[str(atom_global)] = p_dict_list[ind_species][
                    str(ind)
                ]
    else:
        print("uni simple")
        product_unified_dict = p_dict_list[0]
        reactant_unified_dict = r_dict_list[0]

        # for ind, atom_global in enumerate(map_react):
        #    reactant_unified_dict[str(atom_global)] = r_dict_list[0][str(ind)]
        # for ind, atom_global in enumerate(map_prod):
        #    product_unified_dict[str(atom_global)] = p_dict_list[0][str(ind)]

    assert len(reactant_unified_dict) == len(product_unified_dict)

    reactant_unified_dict = dict(sorted(reactant_unified_dict.items()))
    product_unified_dict = dict(sorted(product_unified_dict.items()))
    return reactant_unified_dict, product_unified_dict


def get_composition_info(df_row):
    # helper to get composition info from smiles
    p_mol = Chem.MolFromSmiles(df_row.psmi, sanitize=False)
    r_mol = Chem.MolFromSmiles(df_row.rsmi, sanitize=False)

    # get elements in each molecule
    p_elements = [atom.GetSymbol() for atom in p_mol.GetAtoms()]
    r_elements = [atom.GetSymbol() for atom in r_mol.GetAtoms()]

    # make dictionary with element and counts
    p_dict = {}
    r_dict = {}
    for element in p_elements:
        if element in p_dict:
            p_dict[element] += 1
        else:
            p_dict[element] = 1
    for element in r_elements:
        if element in r_dict:
            r_dict[element] += 1
        else:
            r_dict[element] = 1

    # sort dictionaries
    p_dict = dict(sorted(p_dict.items()))
    r_dict = dict(sorted(r_dict.items()))
    # assert that the elements and values are the same in both reactants and products
    assert (
        p_dict == r_dict
    ), "reactants and products should have same elements and counts"
    natoms = len(p_elements)

    comp_dict = {
        "natoms": natoms,
        "comp_r": r_dict,
        "comp_p": p_dict,
        "elements": sorted(list(set(r_elements))),
        "composition": p_dict,
        "nelements": len(list(set(r_elements))),
    }
    return comp_dict


def construct_bondnet_row(
    df_row,
    prefix_log,
    log_folder_root,
    zero_padding_folder,
    labels=["dE0", "dHrxn298"],
    parser="qchem",
    single_file=False,
):
    # helper that merges all the above functions to construct a bondnet row
    # items for dataframe:
    # reactant_bonds [x]
    # product_bonds [x]
    # reactants_bonds_nometal [x]
    # products_bonds_nometal [x]
    # product_molecule_graph [x]
    # reactant_molecule_graph [x]
    # reaction_id [x]
    # reactant_id [x]
    # product_id [x]
    # composition [x]
    # nelements [x]
    # elements [x]
    # natoms [x]
    # labels - dg
    # label - barrier

    id_reaction = df_row.idx
    id_reactant = hash(df_row.rsmi)
    id_product = hash(df_row.psmi)

    # gets bond list for reactants and products
    (r_bond_list, p_bond_list, bonds_broken, bonds_formed) = get_global_bond_list(
        df_row
    )
    # gets pmg molecule graph for products and reactants
    r_molecule_graph, p_molecule_graph = construct_pmg(
        df_row,
        r_bond_list,
        p_bond_list,
        prefix_log,
        log_folder_root,
        zero_padding_folder,
        parser=parser,
        single_file=single_file,
    )

    # gets composition info
    comp_dict = get_composition_info(df_row)

    # gets labels, if present in row
    label_dict = {}
    for label in labels:
        if label in df_row:
            label_dict[label] = df_row[label]

    # construct bondnet-compatible row
    bondnet_row = {
        "reactant_bonds": r_bond_list,
        "product_bonds": p_bond_list,
        "reactant_bonds_nometal": r_bond_list,
        "product_bonds_nometal": p_bond_list,
        "bonds_broken": bonds_broken,
        "bonds_formed": bonds_formed,
        "product_molecule_graph": p_molecule_graph.as_dict(),
        "reactant_molecule_graph": r_molecule_graph.as_dict(),
        "reaction_id": id_reaction,
        "reactant_id": id_reactant,
        "product_id": id_product,
        "composition": comp_dict["composition"],
        "nelements": comp_dict["nelements"],
        "elements": comp_dict["elements"],
        "natoms": comp_dict["natoms"],
    }
    if label_dict != {}:
        for label in label_dict:
            bondnet_row[label] = label_dict[label]
    else:
        print("Note: no labels found in row")
    # print(bondnet_row.keys())
    return bondnet_row

In [65]:
def green_to_bondnet(
    input_file,
    log_folder_root,
    prefix_log,
    zero_padding_folder,
    labels=["dE0", "dHrxn298"],
    default_charge=None,
    save_path=None,
    parser="qchem",
    single_file=False,
):
    """
    Takes:
        input_file: path to green dataset csv file
        log_folder_root: path to folder containing qm log files
        prefix_log: prefix of qm log files
        zero_padding_folder: number of zeros to pad folder names with
        labels: list of labels to include in bondnet row
        save_path: path to save bondnet-compatible dataframe
    Returns:
        bondnet_df: dataframe with bondnet-compatible rows
    """
    df_input = pd.read_csv(input_file)
    bondnet_rows = []
    for ind, row in df_input.iterrows():
        print("reaction index: ", ind)
        bondnet_row = construct_bondnet_row(
            df_row=row,
            labels=labels,
            prefix_log=prefix_log,
            log_folder_root=log_folder_root,
            zero_padding_folder=zero_padding_folder,
            parser=parser,
            single_file=single_file,
        )
        if default_charge is not None:
            bondnet_row["charge"] = default_charge
        bondnet_rows.append(bondnet_row)

    bondnet_df = pd.DataFrame(bondnet_rows)

    if save_path is not None:
        bondnet_df.to_json(save_path)

    return bondnet_df


input_file = "/home/santiagovargas/Documents/green_2022/ccsdtf12_dz.csv"
log_folder_root = "/home/santiagovargas/Documents/green_2022/ccsdtf12_dz/qm_logs/"
prefix_log = "rxn0"
zero_padding_folder = 5
default_charge = 0
labels = ["dE0", "dHrxn298"]
#labels = ["ea", "dh"]

bondnet_df = green_to_bondnet(
    input_file,
    log_folder_root,
    prefix_log,
    zero_padding_folder,
    default_charge=default_charge,
    labels=labels,
    parser="qchem",
    save_path="../../dataset/green/2022/ccsdtf12_dz.json",
    single_file=False,
)

reaction index:  0
number of log files found: 2


IndexError: list index out of range

In [51]:
! ls /home/santiagovargas/Documents/green_2022/b97d3/qm_logs/rxn000003/
#rxn000000
#rxn000000
#! ls /home/santiagovargas/Documents/green_2022/wb97xd3/rxn000003/qm_logs/

p000003_0.log  p000003_1.log  r000003.log  ts000003.log


In [62]:
! ls /home/santiagovargas/Documents/green_2022/ccsdtf12_dz/qm_logs/rxn010237/

p010237.log  r010237.log  ts010237.log


In [14]:
import pandas as pd
#! ls ../../dataset/green/2022/wb97xd3.json
input_file = "/home/santiagovargas/Documents/green_2022/wb97xd3.csv"
df_input = pd.read_csv(input_file)

In [66]:
check_out = pd.read_json("../../dataset/green/2022/ccsdtf12_dz.json")

In [67]:
check_out

,reactant_bonds,product_bonds,reactant_bonds_nometal,product_bonds_nometal,bonds_broken,bonds_formed,product_molecule_graph,reactant_molecule_graph,reaction_id,reactant_id,product_id,composition,nelements,elements,natoms,dE0,dHrxn298,charge
0,"[[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [0, 6...","[[0, 1], [1, 2], [2, 3], [1, 5], [4, 5], [0, 6...","[[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [0, 6...","[[0, 1], [1, 2], [2, 3], [1, 5], [4, 5], [0, 6...","[[3, 4]]",[],"{'@module': 'pymatgen.analysis.graphs', '@clas...","{'@module': 'pymatgen.analysis.graphs', '@clas...",0,1773502189702824734,-7433491151787688499,"{'C': 2, 'H': 3, 'N': 3, 'O': 1}",4,"[C, H, N, O]",9,48.61085,26.77621,0
1,"[[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [0, 6...","[[0, 2], [1, 2], [1, 5], [4, 5], [3, 4], [0, 6...","[[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [0, 6...","[[0, 2], [1, 2], [1, 5], [4, 5], [3, 4], [0, 6...","[[0, 1], [2, 3]]","[[0, 2]]","{'@module': 'pymatgen.analysis.graphs', '@clas...","{'@module': 'pymatgen.analysis.graphs', '@clas...",1,1773502189702824734,3512967294638128689,"{'C': 2, 'H': 3, 'N': 3, 'O': 1}",4,"[C, H, N, O]",9,74.02980,28.79099,0
2,"[[0, 1], [1, 2], [2, 3], [3, 4], [4, 12], [3, ...","[[0, 5], [0, 6], [0, 1], [1, 2], [2, 8], [2, 9...","[[0, 1], [1, 2], [2, 3], [3, 4], [4, 12], [3, ...","[[0, 5], [0, 6], [0, 1], [1, 2], [2, 8], [2, 9...","[[3, 4], [0, 7]]","[[4, 7], [0, 3]]","{'@module': 'pymatgen.analysis.graphs', '@clas...","{'@module': 'pymatgen.analysis.graphs', '@clas...",2,5179974334361622933,6882518340455797981,"{'C': 3, 'H': 8, 'O': 2}",3,"[C, H, O]",13,97.42200,12.60220,0
3,"[[0, 1], [1, 2], [2, 3], [3, 4], [4, 12], [3, ...","[[0, 1], [1, 12], [0, 5], [0, 6], [0, 7], [2, ...","[[0, 1], [1, 2], [2, 3], [3, 4], [4, 12], [3, ...","[[0, 1], [1, 12], [0, 5], [0, 6], [0, 7], [2, ...","[[1, 2], [4, 12]]","[[1, 12], [2, 4]]","{'@module': 'pymatgen.analysis.graphs', '@clas...","{'@module': 'pymatgen.analysis.graphs', '@clas...",3,5179974334361622933,1795896888664968122,"{'C': 3, 'H': 8, 'O': 2}",3,"[C, H, O]",13,75.25375,28.98589,0
4,"[[0, 1], [1, 2], [2, 3], [3, 4], [4, 12], [3, ...","[[0, 1], [1, 12], [0, 5], [0, 6], [0, 7], [2, ...","[[0, 1], [1, 2], [2, 3], [3, 4], [4, 12], [3, ...","[[0, 1], [1, 12], [0, 5], [0, 6], [0, 7], [2, ...","[[1, 2], [4, 12], [3, 10]]","[[1, 12], [2, 10]]","{'@module': 'pymatgen.analysis.graphs', '@clas...","{'@module': 'pymatgen.analysis.graphs', '@clas...",4,5179974334361622933,5075695619881973540,"{'C': 3, 'H': 8, 'O': 2}",3,"[C, H, O]",13,72.16356,1.41779,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11921,"[[0, 1], [1, 2], [2, 11], [1, 3], [3, 4], [4, ...","[[0, 1], [1, 3], [3, 4], [4, 5], [5, 6], [5, 1...","[[0, 1], [1, 2], [2, 11], [1, 3], [3, 4], [4, ...","[[0, 1], [1, 3], [3, 4], [4, 5], [5, 6], [5, 1...","[[1, 2], [1, 10]]","[[2, 10]]","{'@module': 'pymatgen.analysis.graphs', '@clas...","{'@module': 'pymatgen.analysis.graphs', '@clas...",11956,7235335130296819662,-3738239911298683022,"{'C': 4, 'H': 8, 'O': 3}",3,"[C, H, O]",15,75.56813,79.63518,0
11922,"[[0, 1], [1, 2], [2, 11], [1, 3], [3, 4], [4, ...","[[0, 1], [1, 10], [1, 2], [2, 5], [5, 6], [6, ...","[[0, 1], [1, 2], [2, 11], [1, 3], [3, 4], [4, ...","[[0, 1], [1, 10], [1, 2], [2, 5], [5, 6], [6, ...","[[2, 11]]","[[2, 5], [6, 11]]","{'@module': 'pymatgen.analysis.graphs', '@clas...","{'@module': 'pymatgen.analysis.graphs', '@clas...",11957,7235335130296819662,-3344430078188819697,"{'C': 4, 'H': 8, 'O': 3}",3,"[C, H, O]",15,42.41621,5.79695,0
11923,"[[0, 1], [1, 2], [2, 11], [1, 3], [3, 4], [4, ...","[[0, 1], [1, 2], [2, 11], [1, 3], [3, 4], [3, ...","[[0, 1], [1, 2], [2, 11], [1, 3], [3, 4], [4, ...","[[0, 1], [1, 2], [2, 11], [1, 3], [3, 4], [3, ...","[[4, 5], [3, 12]]","[[5, 12]]","{'@module': 'pymatgen.analysis.graphs', '@clas...","{'@module': 'pymatgen.analysis.graphs', '@clas...",11958,7235335130296819662,4056567741983458091,"{'C': 4, 'H': 8, 'O': 3}",3,"[C, H, O]",15,72.75039,30.54744,0
11924,"[[0

In [6]:
import pandas as pd

In [7]:
green_2020_df = pd.read_json("../../dataset/20231110_parse/green/2020/b97d3_qtaim_12142023.json")
green_2020_df["reactant_bonds_nometal"] = green_2020_df["reactant_bonds"]
green_2020_df["product_bonds_nometal"] = green_2020_df["product_bonds"]
# overwrite 
green_2020_df.to_json("../../dataset/20231110_parse/green/2020/b97d3_qtaim_12142023.json")
# get 

In [ ]:
green_2022_df = pd.read_json("../../dataset/20231110_parse/green/2022/b97d3_qtaim_121423.json")
green_2022_df["reactant_bonds_nometal"] = green_2022_df["reactant_bonds"]
green_2022_df["product_bonds_nometal"] = green_2022_df["product_bonds"]
# overwrite
green_2022_df.to_json("../../dataset/20231110_parse/green/2022/b97d3_qtaim_121423.json")

In [10]:
#green_2022_df_1000 = green_2022_df.head(1000)
# get rows 1000-2000
green_2022_df_1000 = green_2022_df.iloc[5000:7000]
#5000:7000
green_2022_df_1000.to_json("../../dataset/20231110_parse/green/2022/b97d3_qtaim_121423_1000.json")

In [12]:
label_list = green_2022_df.dE0.tolist()
# check if all labels are present
for i in label_list:
    if i is None:
        print("found none")
    # check if a label is a list 
    if isinstance(i, list):
        print("found list")
    # check if anything isn't a float 
    if not isinstance(i, float):
        print("found not float")

In [14]:
label_list[3000:5000]

[72.27154,
 71.21245,
 74.05326,
 53.55321,
 53.36461,
 81.29051,
 93.97196,
 96.11191,
 69.97961,
 104.32752,
 82.61992,
 83.27788,
 92.78960000000001,
 85.63236,
 109.09909,
 65.61033,
 80.90157,
 82.2254,
 69.16053,
 68.20374,
 74.51253,
 83.18144,
 65.40101,
 60.22185,
 89.22438,
 66.67845,
 72.72574,
 59.2227,
 47.97234,
 95.52498,
 62.857730000000004,
 78.98411,
 95.61235,
 45.81801,
 63.31152,
 91.9134,
 95.98456,
 66.90415,
 71.35985,
 101.90894,
 63.88538,
 73.44075,
 61.96778,
 73.23498,
 76.24449,
 33.96381,
 61.6754,
 78.93783,
 42.1056,
 57.15551,
 42.44991,
 105.81333,
 67.06912,
 29.23329,
 50.96015,
 73.72411,
 69.66141,
 56.43339,
 42.77695,
 50.96418,
 55.79417,
 50.96222,
 88.93016,
 60.09474,
 67.55538,
 76.53796,
 65.07786,
 57.79585,
 119.7658,
 59.77795,
 96.5281,
 70.57256,
 64.69838,
 64.76239,
 65.17383,
 71.38817,
 75.75165,
 68.38266,
 62.51367,
 69.12535,
 72.92373,
 61.42265,
 73.85939,
 74.76412,
 55.1195,
 114.20547,
 69.64822,
 56.28944,
 56.91731,
 69.

In [12]:
print(list(green_2022_df.columns))

['reactant_bonds', 'product_bonds', 'reactant_bonds_nometal', 'product_bonds_nometal', 'bonds_broken', 'bonds_formed', 'product_molecule_graph', 'reactant_molecule_graph', 'reaction_id', 'reactant_id', 'product_id', 'composition', 'nelements', 'elements', 'natoms', 'dE0', 'dHrxn298', 'charge', 'extra_feat_atom_reactant_Lagrangian_K', 'extra_feat_atom_product_Lagrangian_K', 'extra_feat_atom_reactant_Hamiltonian_K', 'extra_feat_atom_product_Hamiltonian_K', 'extra_feat_atom_reactant_energy_density', 'extra_feat_atom_product_energy_density', 'extra_feat_atom_reactant_lap_e_density', 'extra_feat_atom_product_lap_e_density', 'extra_feat_atom_reactant_e_loc_func', 'extra_feat_atom_product_e_loc_func', 'extra_feat_atom_reactant_ave_loc_ion_E', 'extra_feat_atom_product_ave_loc_ion_E', 'extra_feat_atom_reactant_delta_g_promolecular', 'extra_feat_atom_product_delta_g_promolecular', 'extra_feat_atom_reactant_delta_g_hirsh', 'extra_feat_atom_product_delta_g_hirsh', 'extra_feat_atom_reactant_esp_nuc